In [ ]:
#import python modules
from snappy import ProductIO, GPF, HashMap
from glob import iglob
from zipfile import ZipFile
import os
import pandas as pd
import numpy as np
import subprocess
#import matplotlib.colors as colors
import matplotlib.image as mpimg
import matplotlib.pyplot as plt

#import jpy
from IPython.core.display import display


pd.options.display.max_colwidth = 80

In [ ]:
#how to call gpt from command line
#print(subprocess.Popen(['gpt', '-h', 'Subset'], stdout=subprocess.PIPE, universal_newlines=True).communicate()[0])


In [ ]:
#HashMap = snappy.jpy.get_type('java.util.HashMap')
# Get snappy Operators
GPF.getDefaultInstance().getOperatorSpiRegistry().loadOperatorSpis()

source_path = r'D:\School\Strathclyde\EF_900\Data\SentinelSatData\Rice\S1\prev'
dest_path = r'D:\School\Strathclyde\EF_900\Data\SentinelSatData\Rice\S1\prev\preprocessed\\'


In [ ]:
def display_product_preview(product, band, min_vv, max_vv, min_vh, max_vh):
    band_data_list = []
    
    for i in band:
        band = product.getBand(i)
        w = band.getRasterWidth()
        h = band.getRasterHeight()
        band_data = np.zeros(w * h, np.float32)
        band.readPixels(0, 0, w, h, band_data)
        band_data.shape = h, w
        band_data_list.append(band_data)
        
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 16))
    ax1.imshow(band_data_list[0], cmap='gray', vmin=min_vv, vmax=max_vv)
    ax1.set_title(output_bands[0])
    ax2.imshow(band_data_list[1], cmap='gray', vmin=min_vh, vmax=max_vh)
    ax2.set_title(output_bands[1])
    
    for ax in fig.get_axes():
        ax.label_outer()

In [ ]:
def write_product(data, file_path, format=None):
    # allowed format are
    # GeoTIFF-BigTIFF,HDF5,Snaphu,BEAM-DIMAP,GeoTIFF+XML,PolSARPro,NetCDF-CF,NetCDF-BEAM,ENVI,JP2,Generic Binary BSQ,Gamma,CSV,NetCDF4-CF,GeoTIFF,NetCDF4-BEAM
    ProductIO.writeProduct(data, file_path, format if format else 'BEAM-DIMAP')


In [ ]:
def create_product_subset()

In [ ]:
def apply_orbit_file(data, datestamp):
    params = HashMap()
    orbit = GPF.createProduct('Apply-Orbit-File', params, data)
    # write_product(orbit, os.path.join(dest_path, '{}_Orb'.format(datestamp)))
    return orbit


In [ ]:
def do_calibration(orbit, datestamp):
    params = HashMap()
    params.put('outputSigmaBand', False)
    params.put('outputGammaBand', False)
    params.put('outputBetaBand', True)
    calibration = GPF.createProduct('Calibration', params, orbit)
    # write_product(calibration, os.path.join(dest_path, '{}_Orb_Cal'.format(datestamp)))
    return calibration


In [ ]:
def perform_multilook(calibration, datestamp, range_look_number=3, azimuth_look_number=3):
    params = HashMap()
    params.put('nRgLooks', range_look_number)
    params.put('nAzLooks', azimuth_look_number)
    params.put('outputIntensity', True)
    params.put('grSquarePixel', True)
    multilook = GPF.createProduct('Multilook', params, calibration)
    # write_product(multilook, os.path.join(dest_path, '{}_Orb_Cal_ML'.format(datestamp)))
    return multilook


In [ ]:
def perform_terrain_flattening(multilook, datestamp):
    params = HashMap()
    params.put('demName', 'SRTM 1Sec HGT')
    params.put('demResamplingMethod', 'BICUBIC_INTERPOLATION')
    params.put('oversamplingMultiple', 1.5)
    params.put('additionalOverlap', 0.1)
    terrain = GPF.createProduct('Terrain-Flattening', params, multilook)
    # write_product(terrain, os.path.join(dest_path, '{}_Orb_Cal_ML_TF'.format(datestamp)))
    return terrain

In [ ]:
def dem_coregistration(terrain, datestamp):
    params = HashMap()
    params.put('demName', 'SRTM 1Sec HGT')
    params.put('demResamplingMethod', 'BICUBIC_INTERPOLATION')
    # not sure if BILINEAR_INTERPOLATION would produce anything different
    # worth checking
    params.put('resamplingType', 'BICUBIC_INTERPOLATION')
    params.put('tileExtensionPercent', 100)
    params.put('maskOutAreaWithoutElevation', True)
    coregistered = GPF.createProduct('DEM-Assisted-Coregistration', params, terrain)
    write_product(coregistered, os.path.join(dest_path, '{}_Orb_Cal_ML_TF_Stack'.format(datestamp)))
    return coregistered

In [ ]:
def speckle_reduction(data, datestamp):
    params = HashMap()
    params.put('filter', 'Lee Sigma')
    params.put('enl', 4.0)
    params.put('numLooksStr', '4')
    params.put('windowSize', '9x9')
    params.put('sigmaStr', '0.9')
    params.put('targetWindowSizeStr', '5x5')
    speckle = GPF.createProduct('Speckle-Filter', params, data)
    # write_product(speckle, os.path.join(dest_path, '{}_Orb_Cal_ML_TF_Stack_Spk'.format(datestamp)))
    return speckle

In [ ]:
def ellipsoid_correction(speckle, datestamp):
    params = HashMap()
    params.put('imgResamplingMethod', 'BILINEAR_INTERPOLATION')
    params.put('mapProjection', 'WGS84(DD)')
    ec = GPF.createProduct('Ellipsoid-Correction-GG', params, speckle)
    write_product(ec, os.path.join(dest_path, '{}_Orb_Cal_ML_TF_Stack_Spk_EC'.format(datestamp)))
    write_product(ec, os.path.join(dest_path, '{}_Orb_Cal_ML_TF_Stack_Spk_EC'.format(datestamp)), format='GeoTIFF')
    return ec

In [ ]:
def process(file):
    data = ProductIO.readProduct(os.path.join(input, file))
    # get the end date from the file name and get first 8 substring as YYYYmmdd
    datestamp = file.split('_')[4][:8]
    orbit = apply_orbit_file(data, datestamp)
    calibration = do_calibration(orbit, datestamp)
    multilook = perform_multilook(calibration, datestamp)
    terrain = perform_terrain_flattening(multilook, datestamp)
    # coregistered = dem_coregistration(terrain, datestamp)
    speckle = speckle_reduction(terrain, datestamp)
    final = ellipsoid_correction(speckle, datestamp)
    print('finished')
    return True

In [ ]:
def set_path():
    os.path.dirname(os.path.dirname(__file__))
    path = os.path.join(os.getcwd())
    os.chdir(path)
    return path

In [ ]:
def main():
    path = set_path()
    files = [f for f in os.listdir(source_path) if f.endswith('.zip')]
    for file in files:
        status = process(file)


if __name__ == '__main__':
    main()